<a href="https://colab.research.google.com/github/arssite/GENAi-Assessment/blob/main/Sentiment_Analyzer_Using_DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch gradio pandas matplotlib openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.8 MB/s eta 0:00:00


In [8]:
!pip install python-docx PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.3 MB/s eta 0:00:00


In [12]:
import torch
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
from transformers import pipeline
from docx import Document
from PyPDF2 import PdfReader

# Initialize the sentiment analysis pipeline
analyzer = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# Function to analyze sentiment for a single sentence
def sentiment_analyzer(review):
    sentiment = analyzer(review)
    return sentiment[0]['label']

# Create a pie chart visualization for the sentiment
def sentiment_pie_chart(sentiment_labels):
    sentiment_counts = pd.Series(sentiment_labels).value_counts()

    fig, ax = plt.subplots()
    sentiment_counts.plot(kind='pie', autopct='%1.1f%%', colors=['green', 'red'], ax=ax)
    ax.set_ylabel('')
    ax.set_title('Sentiment Distribution')

    return fig

# Function to analyze a single input sentence
def analyze_single_sentence(sentence):
    sentiment = sentiment_analyzer(sentence)
    return f"The sentiment of the sentence is: {sentiment}"

# Function to read and analyze reviews from Excel, PDF, or DOCX files
def read_reviews_and_analyze_sentiment(file_object):
    if file_object.name.endswith('.xlsx'):
        # Load the Excel file into a DataFrame
        df = pd.read_excel(file_object)
        if 'Reviews' not in df.columns:
            raise ValueError("Excel file must contain a 'Reviews' column.")
        reviews = df['Reviews'].tolist()

    elif file_object.name.endswith('.docx'):
        # Read the content of the DOCX file
        doc = Document(file_object)
        reviews = [para.text for para in doc.paragraphs if para.text.strip()]

    elif file_object.name.endswith('.pdf'):
        # Read the content of the PDF file
        reader = PdfReader(file_object)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        reviews = text.split('\n')  # Assuming reviews are newline-separated

    else:
        raise ValueError("Unsupported file format. Please upload .xlsx, .pdf, or .docx files.")

    # Analyze the sentiment of each review
    sentiments = [sentiment_analyzer(review) for review in reviews]
    df = pd.DataFrame({'Reviews': reviews, 'Sentiment': sentiments})

    # Generate pie chart
    chart_object = sentiment_pie_chart(sentiments)

    return df, chart_object

# Gradio interface combining single sentence analysis and file-based review sentiment analysis
def main_interface(input_option, sentence=None, file=None):
    if input_option == "Single Sentence":
        if sentence:
            result = analyze_single_sentence(sentence)
            return None, None, result  # Single sentence output
        else:
            return None, None, "Please enter a sentence."
    elif input_option == "File Upload":
        if file:
            df, chart_object = read_reviews_and_analyze_sentiment(file)
            return df, chart_object, None  # File output
        else:
            return None, None, "Please upload a file."

# Gradio interface
demo = gr.Interface(
    fn=main_interface,
    inputs=[
        gr.Radio(label="Choose Input Type", choices=["Single Sentence", "File Upload"], value="Single Sentence"),
        gr.Textbox(label="Enter a sentence for sentiment analysis (if selected)", placeholder="Type your sentence here..."),
        gr.File(file_types=["xlsx", "pdf", "docx"], label="Upload your review comment file (if selected)")
    ],
    outputs=[
        gr.Dataframe(label="Sentiment Analysis Results (For File Uploads)"),
        gr.Plot(label="Sentiment Distribution Chart (For File Uploads)"),
        gr.Textbox(label="Single Sentence Sentiment Result (For Single Sentence Input)")
    ],
    title="@GenAILearniverse Project 3: Sentiment Analyzer",
    description="This application analyzes the sentiment of either a single sentence or reviews in uploaded files (Excel, PDF, DOCX)."
)

demo.launch()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0d9c320d3dab444d6c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [14]:
!gradio deploy


mnist_test.csv:   0% 0.00/18.3M [00:00<?, ?B/s]
mnist_train_small.csv:   0% 0.00/36.5M [00:00<?, ?B/s]

mnist_test.csv:   3% 541k/18.3M [00:00<00:03, 5.09MB/s]
mnist_train_small.csv:   2% 606k/36.5M [00:00<00:06, 5.74MB/s]
mnist_test.csv:  24% 4.42M/18.3M [00:00<00:00, 20.5MB/s]
mnist_test.csv:  59% 10.8M/18.3M [00:00<00:00, 27.8MB/s]
mnist_test.csv:  88% 16.0M/18.3M [00:00<00:00, 17.9MB/s]
mnist_train_small.csv:  46% 16.6M/36.5M [00:00<00:01, 15.4MB/s]
mnist_train_small.csv:  62% 22.8M/36.5M [00:01<00:00, 23.6MB/s]
mnist_test.csv: 100% 18.3M/18.3M [00:01<00:00, 12.6MB/s]


Upload 2 LFS files:  50% 1/2 [00:01<00:01,  1.66s/it]
mnist_train_small.csv: 100% 36.5M/36.5M [00:02<00:00, 13.2MB/s]


Upload 2 LFS files: 100% 2/2 [00:02<00:00,  1.48s/it]
Space available at https://huggingface.co/spaces/arssite/Sentiment_Analyzer_Using_Distilbert
